In [1]:
import os
import backtrader as bt
import numpy as np
from gym import spaces

from btgym import BTgymEnv, BTgymDataset
from btgym.strategy.observers import Reward, Position, NormPnL
from btgym.algorithms import Launcher, A3C, PPO, Unreal, Aac1dPolicy

from btgym.research import DevStrat_4_7, DevStrat_4_8, DevStrat_4_9, DevStrat_4_10, DevStrat_4_11
from btgym.research import AacRL2Policy, StackedLstmPolicy, AacStackedRL2Policy

from btgym.datafeed import BTgymRandomTrial, BTgymSequentialTrial

In [22]:
# Set backtesting engine parameters:

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    DevStrat_4_11,
    drawdown_call=10, # max % to loose, in percent of initial cash
    target_call=10,  # max % to win, same
    skip_frame=10,
    gamma=0.99,
    reward_scale=5, # gardients nitrox
)
# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.addsizer(bt.sizers.SizerFix, stake=5000,)  

#MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Visualisations for reward, position and PnL dynamics:
MyCerebro.addobserver(Reward)
MyCerebro.addobserver(Position)
MyCerebro.addobserver(NormPnL)

# Data var. 1: up to seven years of 1 minute bars:
data_m1_7_year = [
    #'../examples/data/DAT_ASCII_EURUSD_M1_2010.csv',
    #'../examples/data/DAT_ASCII_EURUSD_M1_2011.csv',
    #'../examples/data/DAT_ASCII_EURUSD_M1_2012.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_2013.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_2014.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_2015.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
]

# Data var. 2: up to six month of 1 minute bars:
data_m1_6_month = [
    '../examples/data/DAT_ASCII_EURUSD_M1_201701.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_201702.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_201704.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_201705.csv',
    '../examples/data/DAT_ASCII_EURUSD_M1_201706.csv',
]

# Uncomment single choice:
MyDataset = BTgymDataset( #BTgymSequentialTrial(  
    #filename=data_m1_7_year,
    #filename=data_m1_6_month,
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_2015.csv',
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    filename='../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_201704.csv',
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_201703_1_10.csv',  # ten days
    #filename='../examples/data/test_sine_1min_period256_delta0002.csv',  # simple sine 
    #filename='../examples/data/test_bent_sine_1min_period1500_300_delta0002.csv',  # increasing sine freq
    #filename='../examples/data/test_bent_sine_1min_period_300>1500_delta0002.csv',  # decreasing sine freq
    start_weekdays={0, 1, 2, 3, 4, 5, 6},
    episode_duration={'days': 0, 'hours': 23, 'minutes': 55},
    start_00=False,
    time_gap={'hours': 10},
    train_range={'days': 20, 'hours': 0, 'minutes': 0},
    test_range={'days': 10, 'hours': 0, 'minutes': 0},
    train_samples=100000,
    test_samples=10,
    test_period=200,
    trial_start_00=True,
    expanding=False,
    b_alpha=200,
    b_beta=0.8,
    b_anneal_steps=10000
)

env_config = dict(
    class_ref=BTgymEnv, 
    kwargs=dict(
        dataset=MyDataset,
        engine=MyCerebro,
        render_modes=['episode', 'human', 'external', 'internal'],
        render_state_as_image=True,
        render_ylabel='OHL_diff. / Internals',
        render_size_episode=(12,8),
        render_size_human=(9, 4),
        render_size_state=(11, 3),
        render_dpi=75,
        port=5000,
        data_port=4999,
        connect_timeout=60,
        verbose=0,
    )
)

cluster_config = dict(
    host='127.0.0.1',
    port=12230,
    num_workers=6,  # Set according CPU's available or so
    num_ps=1,
    num_envs=1,
    log_dir=os.path.expanduser('~/tmp/test_4_8'),
)

policy_config = dict(
    class_ref=AacStackedRL2Policy,
    kwargs={
        'lstm_layers': (256, 256),
        'lstm_2_init_period': 60,
        'encode_internal_state': False,
        #'dropout_keep_prob': 0.99,
    }
)

trainer_config = dict(
    class_ref=Unreal,
    kwargs=dict(
        opt_learn_rate=[1e-4, 1e-4], # Random log-uniform 
        opt_end_learn_rate=1e-5,
        opt_decay_steps=20*10**6,
        model_gamma=0.99,
        model_gae_lambda=1.0,
        model_beta=[0.1, 0.03], # Entropy reg, random log-uniform
        rollout_length=20,
        time_flat=True,
        use_value_replay=False,
        use_pixel_control=False,
        use_reward_prediction=False,
        rp_reward_threshold=0.1,
        rp_sequence_size=4,
        rp_lambda=0.01,
        model_summary_freq=100,
        episode_summary_freq=5,
        env_render_freq=20,
    )
)

In [ ]:
launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=False,
    max_env_steps=100*10**6,
    root_random_seed=0,
    purge_previous=1,  # ask to override previously saved model and logs
    verbose=0
)

# Train it:
launcher.run()

</Users/muzikin/tmp/test_4_8> already exists. Override[y/n]? y


[2017-12-27 20:25:15,379] Files in </Users/muzikin/tmp/test_4_8> purged.
[2017-12-27 20:25:19,737] AAC_0: learn_rate: 0.000100, entropy_beta: 0.083925


Press `Ctrl-C` or [Kernel]->[Interrupt] to stop training and close launcher.


[2017-12-27 20:25:22,402] AAC_5: learn_rate: 0.000100, entropy_beta: 0.030710
[2017-12-27 20:25:22,402] AAC_1: learn_rate: 0.000100, entropy_beta: 0.044168
[2017-12-27 20:25:22,402] AAC_3: learn_rate: 0.000100, entropy_beta: 0.082202
[2017-12-27 20:25:22,415] AAC_2: learn_rate: 0.000100, entropy_beta: 0.061766
[2017-12-27 20:25:22,426] AAC_4: learn_rate: 0.000100, entropy_beta: 0.030711


INFO:tensorflow:Starting queue runners.


...

[2017-12-27 20:38:04,618] global/global_step/sec: 230.441


INFO:tensorflow:global/global_step/sec: 233.265


[2017-12-27 20:40:04,623] global/global_step/sec: 233.265


INFO:tensorflow:Saving checkpoint to path /Users/muzikin/tmp/test_4_8/train/model.ckpt


[2017-12-27 20:41:04,607] Saving checkpoint to path /Users/muzikin/tmp/test_4_8/train/model.ckpt
